In [13]:
import pandas as pd
import tensorflow as tf 
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV
#from pandas_profiling import ProfileReport

In [4]:
df = pd.read_csv('all_train.csv')

In [5]:
#Install the below libaries before importing

#EDA using pandas-profiling
#profile = ProfileReport(pd.read_csv('all_train.csv'), explorative=True)

#Saving results to a HTML file
#profile.to_file("output.html")


### EDA is complete in the output file. There is no missing data. The response variable, label, is pretty closeley even, 3500879 to 3499121.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 29 columns):
 #   Column   Dtype  
---  ------   -----  
 0   # label  float64
 1   f0       float64
 2   f1       float64
 3   f2       float64
 4   f3       float64
 5   f4       float64
 6   f5       float64
 7   f6       float64
 8   f7       float64
 9   f8       float64
 10  f9       float64
 11  f10      float64
 12  f11      float64
 13  f12      float64
 14  f13      float64
 15  f14      float64
 16  f15      float64
 17  f16      float64
 18  f17      float64
 19  f18      float64
 20  f19      float64
 21  f20      float64
 22  f21      float64
 23  f22      float64
 24  f23      float64
 25  f24      float64
 26  f25      float64
 27  f26      float64
 28  mass     float64
dtypes: float64(29)
memory usage: 1.5 GB


In [7]:
# change # label to "response"
df.rename(columns ={'# label': 'response'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 29 columns):
 #   Column    Dtype  
---  ------    -----  
 0   response  float64
 1   f0        float64
 2   f1        float64
 3   f2        float64
 4   f3        float64
 5   f4        float64
 6   f5        float64
 7   f6        float64
 8   f7        float64
 9   f8        float64
 10  f9        float64
 11  f10       float64
 12  f11       float64
 13  f12       float64
 14  f13       float64
 15  f14       float64
 16  f15       float64
 17  f16       float64
 18  f17       float64
 19  f18       float64
 20  f19       float64
 21  f20       float64
 22  f21       float64
 23  f22       float64
 24  f23       float64
 25  f24       float64
 26  f25       float64
 27  f26       float64
 28  mass      float64
dtypes: float64(29)
memory usage: 1.5 GB


### Split Data into test and train while scaling

In [12]:
scaler = StandardScaler()

X = df.loc[:, df.columns != 'response'].values

y = df['response'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Grid Search CV

In [ ]:
log_root = os.path.join(os.curdir, "my_training_logs")

def get_log_dir():
    import time
    timestamp = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(log_root, timestamp)

log_dir = get_log_dir()

# # Print current working directory and log directory
# print(os.getcwd())
# print(log_dir)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir)

def create_model(num_hidden_layers=1, num_neurons=30, learning_rate=3e-3, input_shape=[28]):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation="relu"))
        model.add(tf.keras.layers.Dropout(.2, input_shape=(2,)))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.BinaryCrossentropy()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

regressor = tf.keras.wrappers.scikit_learn.KerasRegressor(create_model)

param_distributions = {
    "num_hidden_layers": [0, 1, 2, 3],
    "num_neurons": np.arange(1, 500)
}

early_stop = EarlyStopping(monitor='val_loss', patience=3, min_delta=2e-4)

random_search_cv = RandomizedSearchCV(regressor, param_distributions, n_iter=5, cv=3)
random_search_cv.fit(X_train, y_train, epochs=500,
                      validation_data=(X_test, y_test),
                      callbacks=[tensorboard_callback, early_stop], batch_size=1000)


C:\Users\girar\AppData\Local\Temp\ipykernel_19044\3312735626.py:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  regressor = tf.keras.wrappers.scikit_learn.KerasRegressor(create_model)


Epoch 1/500
3734/3734 [==============================] - 6s 1ms/step - loss: 0.4011 - accuracy: 0.8159 - val_loss: 0.3660 - val_accuracy: 0.8365
Epoch 2/500
3734/3734 [==============================] - 6s 2ms/step - loss: 0.3660 - accuracy: 0.8364 - val_loss: 0.3647 - val_accuracy: 0.8370
Epoch 3/500
3734/3734 [==============================] - 5s 1ms/step - loss: 0.3658 - accuracy: 0.8365 - val_loss: 0.3647 - val_accuracy: 0.8366
Epoch 4/500
3734/3734 [==============================] - 5s 1ms/step - loss: 0.3658 - accuracy: 0.8364 - val_loss: 0.3647 - val_accuracy: 0.8369
Epoch 5/500
58334/58334 [==============================] - 53s 905us/step - loss: 0.3650 - accuracy: 0.8369
Epoch 1/500
3734/3734 [==============================] - 6s 1ms/step - loss: 0.3942 - accuracy: 0.8202 - val_loss: 0.3648 - val_accuracy: 0.8369
Epoch 2/500
3734/3734 [==============================] - 5s 1ms/step - loss: 0.3655 - accuracy: 0.8367 - val_loss: 0.3646 - val_accuracy: 0.8371
Epoch 3/500
3734/3734 

3734/3734 [==============================] - 41s 11ms/step - loss: 0.2637 - accuracy: 0.8818 - val_loss: 0.2604 - val_accuracy: 0.8837
Epoch 20/500
58334/58334 [==============================] - 64s 1ms/step - loss: 0.2607 - accuracy: 0.8834
Epoch 1/500
3734/3734 [==============================] - 42s 11ms/step - loss: 0.2904 - accuracy: 0.8664 - val_loss: 0.2732 - val_accuracy: 0.8758
Epoch 2/500
3734/3734 [==============================] - 40s 11ms/step - loss: 0.2751 - accuracy: 0.8753 - val_loss: 0.2680 - val_accuracy: 0.8795
Epoch 3/500
3734/3734 [==============================] - 40s 11ms/step - loss: 0.2718 - accuracy: 0.8773 - val_loss: 0.2661 - val_accuracy: 0.8799
Epoch 4/500
3734/3734 [==============================] - 40s 11ms/step - loss: 0.2699 - accuracy: 0.8783 - val_loss: 0.2648 - val_accuracy: 0.8809
Epoch 5/500
3734/3734 [==============================] - 40s 11ms/step - loss: 0.2689 - accuracy: 0.8789 - val_loss: 0.2648 - val_accuracy: 0.8810
Epoch 6/500
3734/3734 [

3734/3734 [==============================] - 21s 6ms/step - loss: 0.2749 - accuracy: 0.8760 - val_loss: 0.2707 - val_accuracy: 0.8799
Epoch 11/500
58334/58334 [==============================] - 54s 930us/step - loss: 0.2716 - accuracy: 0.8797
Epoch 1/500
3734/3734 [==============================] - 23s 6ms/step - loss: 0.3035 - accuracy: 0.8591 - val_loss: 0.2799 - val_accuracy: 0.8730
Epoch 2/500
3734/3734 [==============================] - 22s 6ms/step - loss: 0.2844 - accuracy: 0.8702 - val_loss: 0.2733 - val_accuracy: 0.8766
Epoch 3/500
3734/3734 [==============================] - 21s 6ms/step - loss: 0.2808 - accuracy: 0.8725 - val_loss: 0.2714 - val_accuracy: 0.8778
Epoch 4/500
3734/3734 [==============================] - 21s 6ms/step - loss: 0.2789 - accuracy: 0.8737 - val_loss: 0.2700 - val_accuracy: 0.8785
Epoch 5/500
3734/3734 [==============================] - 22s 6ms/step - loss: 0.2775 - accuracy: 0.8746 - val_loss: 0.2697 - val_accuracy: 0.8787
Epoch 6/500
3734/3734 [====

3732/3734 [============================>.] - ETA: 0s - loss: 0.2846 - accuracy: 0.8693